In [ ]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import re
import time
import os
import pandas as pd
from IPython.display import HTML
import time
from datetime import datetime
import sys

In [ ]:
####mount your colab drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [ ]:
#@title Create the function that will look up news articles for one stock at a time. Just enter the function name "stock_news_collect()" and then follow the prompts.
def stock_news_collect():
    stock=input("Enter the name or ticker of the stock you would like to collect news articles about: ")

    tmp_query=f'https://www.investing.com/search/?q={stock}'

    tempqueryhtml=requests.get(tmp_query)
    query_page=BeautifulSoup(tempqueryhtml.text,'html.parser')
    query_text=query_page.find(class_='js-inner-all-results-quotes-wrapper newResultsContainer quatesTable')
    query_table=query_text.find_all(class_='js-inner-all-results-quote-item row')

    country, ticker, full_name, exchange, link = [],[],[],[],[]
    for i in range(0,len(query_table)):
        tmp_country=re.findall('(?<=ceFlags middle )(.*)(?="></i></span>)',str(query_table[i]))
        country.append(tmp_country[0])

        tmp_tick=re.findall('(?<= class="second">)(.*)(?=</span>)',str(query_table[i]))
        ticker.append(tmp_tick[0])

        tmp_name=re.findall('(?<= class="third">)(.*)(?=</span>)',str(query_table[i]))
        if "&amp;" in tmp_name[0]:
          tmp_name[0]=re.sub('&amp;','&',tmp_name[0])
        full_name.append(tmp_name[0])

        tmp_ex=re.findall('(?<= class="fourth">)(.*)(?=</span>)',str(query_table[i]))
        exchange.append(tmp_ex[0])

        part_link=re.findall('(?<=href=")(.*)(?=">)',str(query_table[i]))
        full_link="https://www.investing.com" + part_link[0]
        link.append(full_link)

    tmp_stock_df=pd.DataFrame()
    tmp_stock_df['country']=country
    tmp_stock_df['ticker']=ticker
    tmp_stock_df['full_name']=full_name
    tmp_stock_df['exchange']=exchange
    tmp_stock_df['link']=link

    display(HTML(tmp_stock_df.to_html(render_links=True, escape=False)))

    selection=input(f"Confirm the number index of the stock you wanted news articles for (0-{max(tmp_stock_df.index)}): ")

    stock_link=tmp_stock_df.loc[int(selection),'link']+'-news'

    tempstockhtml=requests.get(stock_link)
    stock_page=BeautifulSoup(tempstockhtml.text,'html.parser')

    stock_page_index=stock_page.find_all(class_='flex gap-2 items-center')
    stock_page_count=stock_page_index[0].find_all(class_='flex rounded leading-5 border font-semibold items-center border-[#F7F7F8] bg-[#F7F7F8] text-[#1256A0] p-[11px]')
    page_count_len=int(len(stock_page_count))-1
    total_page_count=int(stock_page_count[page_count_len].text)

    pages=input(f'How many pages worth of articles would you like to collect (1-{total_page_count}):')

    full_news_DF=pd.DataFrame()
    for pc in range(0,int(pages)):
      pages_1=pc+1
      stock_link=tmp_stock_df.loc[int(selection),'link']+'-news/'+str(pages_1)
      tempstockhtml=requests.get(stock_link)
      stock_page=BeautifulSoup(tempstockhtml.text,'html.parser')
      stock_text=stock_page.find_all(class_='block w-full sm:flex-1')

      art_name, art_source, art_date, art_link, art_text = [],[],[],[],[]
      for i in range(0,len(stock_text)):
        if 'mt-2.5' not in str(stock_text[i]):
          temp_name=stock_text[i].find(class_='inline-block text-sm leading-5 sm:text-base sm:leading-6 md:text-lg md:leading-7 font-bold mb-2 hover:underline')
          art_name.append(temp_name.text)

          tmp_source=re.findall('(?<="news-provider-name">)(.*)(?=</span></li>)',str(stock_text[i]))
          art_source.append(tmp_source[0])

          temp_link=stock_text[i].find(class_='inline-block text-sm leading-5 sm:text-base sm:leading-6 md:text-lg md:leading-7 font-bold mb-2 hover:underline')
          part_link=re.findall('(?<=href=")(.*)(?=">)',str(temp_link))
          full_link="https://www.investing.com" + part_link[0]
          art_link.append(full_link)

          tempnewshtml=requests.get(full_link)
          news_page=BeautifulSoup(tempnewshtml.text,'html.parser')

          date_find=news_page.find_all(class_='contentSectionDetails')
          for date in range(0,len(date_find)):
            if "Published" in date_find[date].text:
              temp_date=re.findall('(?<=Published )(.*)(?=</span>)',str(date_find[date]))
              temp_date=temp_date[0].replace(' ET','')
              datetime_object = datetime.strptime(temp_date, '%b %d, %Y %I:%M%p')
          art_date.append(datetime_object)

          news_body=news_page.find(class_='WYSIWYG articlePage')
          news_text=news_body.find_all('p')
          tmp_str=str()
          for i in range(0,len(news_text)):
            if "Position added successfully to:" not in news_text[i].text:
              tmp_str=tmp_str+' ' + news_text[i].text
          art_text.append(tmp_str)
          print('Collected the text for an article. Now pausing for 15 seconds...')
          time.sleep(15)

      tmp_news_df=pd.DataFrame()
      tmp_news_df['art_name']=art_name
      tmp_news_df['art_source']=art_source
      tmp_news_df['art_date']=art_date
      tmp_news_df['art_link']=art_link
      tmp_news_df['art_text']=art_text

      temp_comb_DF=[full_news_DF,tmp_news_df]
      full_news_DF=pd.concat(temp_comb_DF)

      display(f"Just finished page {pages_1}.")

    full_news_DF=full_news_DF.reset_index(drop=True)

    return HTML(full_news_DF.to_html(render_links=True, escape=False))

In [ ]:
#@title Create the function for looking up articles for multiple stocks at once: stock_news_collect_list(stock_list,pages). "stock_list" input should be a list of stock names or tickers. "pages" input should be how many pages worth of articles you'd like to collect for each stock.
def stock_news_collect_list(stock_list,pages):
  full_news_DF=pd.DataFrame()
  fail_list=[]
  for stock in stock_list:
    try:
      tmp_query=f'https://www.investing.com/search/?q={stock}'
      tempqueryhtml=requests.get(tmp_query)
      query_page=BeautifulSoup(tempqueryhtml.text,'html.parser')
      query_text=query_page.find(class_='js-inner-all-results-quotes-wrapper newResultsContainer quatesTable')
      query_table=query_text.find_all(class_='js-inner-all-results-quote-item row')
      part_link=re.findall('(?<=href=")(.*)(?=">)',str(query_table[0]))
      stock_link="https://www.investing.com" + part_link[0] + '-news/'

      tmp_country=re.findall('(?<=ceFlags middle )(.*)(?="></i></span>)',str(query_table[0]))
      tmp_tick=re.findall('(?<= class="second">)(.*)(?=</span>)',str(query_table[0]))
      tmp_name=re.findall('(?<= class="third">)(.*)(?=</span>)',str(query_table[0]))
      if "&amp;" in tmp_name[0]:
        tmp_name[0]=re.sub('&amp;','&',tmp_name[0])
      tmp_ex=re.findall('(?<= class="fourth">)(.*)(?=</span>)',str(query_table[0]))

      tempstockhtml=requests.get(stock_link)
      stock_page=BeautifulSoup(tempstockhtml.text,'html.parser')

      stock_page_index=stock_page.find_all(class_='flex gap-2 items-center')
      stock_page_count=stock_page_index[0].find_all(class_='flex rounded leading-5 border font-semibold items-center border-[#F7F7F8] bg-[#F7F7F8] text-[#1256A0] p-[11px]')
      page_count_len=int(len(stock_page_count))-1
      total_page_count=int(stock_page_count[page_count_len].text)

      if pages > total_page_count:
        pages=total_page_count

      for pc in range(0,int(pages)):
        pages_1=pc+1
        tmp_stock_link=stock_link
        tmp_stock_link=tmp_stock_link+str(pages_1)
        tempstockhtml=requests.get(tmp_stock_link)
        stock_page=BeautifulSoup(tempstockhtml.text,'html.parser')
        stock_text=stock_page.find_all(class_='block w-full sm:flex-1')

        stock_name,stock_tick,stock_country,stock_ex, art_name, art_source, art_date, art_link, art_text = [],[],[],[],[],[],[],[],[]
        for i in range(0,len(stock_text)):
          if 'mt-2.5' not in str(stock_text[i]):
            stock_name.append(tmp_name[0])
            stock_tick.append(tmp_tick[0])
            stock_country.append(tmp_country[0])
            stock_ex.append(tmp_ex[0])

            temp_name=stock_text[i].find(class_='inline-block text-sm leading-5 sm:text-base sm:leading-6 md:text-lg md:leading-7 font-bold mb-2 hover:underline')
            art_name.append(temp_name.text)

            tmp_source=re.findall('(?<="news-provider-name">)(.*)(?=</span></li>)',str(stock_text[i]))
            art_source.append(tmp_source[0])

            temp_link=stock_text[i].find(class_='inline-block text-sm leading-5 sm:text-base sm:leading-6 md:text-lg md:leading-7 font-bold mb-2 hover:underline')
            part_link=re.findall('(?<=href=")(.*)(?=">)',str(temp_link))
            full_link="https://www.investing.com" + part_link[0]
            art_link.append(full_link)

            tempnewshtml=requests.get(full_link)
            news_page=BeautifulSoup(tempnewshtml.text,'html.parser')

            date_find=news_page.find_all(class_='contentSectionDetails')
            for date in range(0,len(date_find)):
              if "Published" in date_find[date].text:
                temp_date=re.findall('(?<=Published )(.*)(?=</span>)',str(date_find[date]))
                temp_date=temp_date[0].replace(' ET','')
                datetime_object = datetime.strptime(temp_date, '%b %d, %Y %I:%M%p')
            art_date.append(datetime_object)

            news_body=news_page.find(class_='WYSIWYG articlePage')
            news_text=news_body.find_all('p')
            tmp_str=str()
            for i in range(0,len(news_text)):
              if "Position added successfully to:" not in news_text[i].text:
                tmp_str=tmp_str+' ' + news_text[i].text
            art_text.append(tmp_str)
            print(f'Collected the text for an article about {stock}. Now pausing for 15 seconds...')
            time.sleep(15)

        tmp_news_df=pd.DataFrame()
        tmp_news_df['stock_name']=stock_name
        tmp_news_df['stock_tick']=stock_tick
        tmp_news_df['stock_country']=stock_country
        tmp_news_df['stock_ex']=stock_ex
        tmp_news_df['art_name']=art_name
        tmp_news_df['art_source']=art_source
        tmp_news_df['art_date']=art_date
        tmp_news_df['art_link']=art_link
        tmp_news_df['art_text']=art_text

        temp_comb_DF=[full_news_DF,tmp_news_df]
        full_news_DF=pd.concat(temp_comb_DF)

        display(f"Just finished {stock}, page {pages_1}.")

        full_news_DF=full_news_DF.reset_index(drop=True)
        #full_news_DF.to_csv('/content/drive/My Drive/Colab Notebooks/full_news_DF.csv',header=True,index=False)
    except AttributeError:
      print(f'Could not locate a page for {stock}')
      fail_list.append(stock)
      continue
  return full_news_DF, fail_list

In [ ]:
#run the function for collecting articles about one specific stock.
stock_news=stock_news_collect()

Enter the name or ticker of the stock you would like to collect news articles about: hasbro


,country,ticker,full_name,exchange,link
0,USA,HAS,Hasbro Inc,Stock - NASDAQ equities,https://www.investing.com/equities/hasbro-inc
1,Brazil,H1AS34,Hasbro Inc,Stock - B3 equities,https://www.investing.com/equities/hasbro-drc-sa
2,Austria,HAS,Hasbro Inc,Stock - Vienna equities,https://www.investing.com/equities/hasbro-inc?cid=1010943
3,Russian_Federation,HAS-RM,Hasbro Inc,Stock - Moscow equities,https://www.investing.com/equities/hasbro-inc?cid=1180687


Confirm the number index of the stock you wanted news articles for (0-3): 0
How many pages worth of articles would you like to collect (1-55):1
Collected the text for an article. Now pausing for 15 seconds...
Collected the text for an article. Now pausing for 15 seconds...
Collected the text for an article. Now pausing for 15 seconds...
Collected the text for an article. Now pausing for 15 seconds...
Collected the text for an article. Now pausing for 15 seconds...
Collected the text for an article. Now pausing for 15 seconds...
Collected the text for an article. Now pausing for 15 seconds...


'Just finished page 1.'

In [ ]:
#take a look at the info
stock_news

,art_name,art_source,art_date,art_link,art_text
0,Hasbro Q3 results prompt downgrade by BofA analysts,Investing.com,2023-10-27 12:12:00,https://www.investing.com/news/stock-market-news/hasbro-q3-results-prompt-downgrade-by-bofa-analysts-432SI-3212215,"Hasbro (NASDAQ:HAS) was cut to Neutral from Buy at BofA on Friday, with analysts lowering the stock price target to $53 from $90 per share. Analysts said the company's Baldur's Gate and Monopoly Go games are not enough to offset the decline in toys. ""We're downgrading Hasbro to Neutral following the company's 3Q earnings results,"" wrote the analysts. ""We had recently upgraded Hasbro to Buy given our expectation for a strong beat in the Wizards segment (WOTC) on the back of its Lord of the Rings Magic set and royalty revenue from Baldur's Gate 3 and Monopoly Go."" ""HAS did report very strong 3Q WOTC revenue (+38% YoY vs. consensus +26%), but unfortunately, its Consumer Products (CP) sales declined 19% YoY, well below our -13% estimate and consensus -5%,"" they explained. The analysts also said the company did not raise its guidance as the investment bank had expected. In addition, they ""remain bullish on Tim Kilpin’s ability to turn around HAS’ toy business"" but said, ""recent commentary suggests this may happen more in 2025."""
1,Wall Street retreats despite strong Q3 GDP growth and consumer spending surge,Investing.com,2023-10-27 03:04:00,https://www.investing.com/news/stock-market-news/wall-street-retreats-despite-strong-q3-gdp-growth-and-consumer-spending-surge-93CH-3211247,"Wall Street stocks experienced a significant drop on Thursday, despite the US economy boasting a robust 4.9% annualized growth in Q3 and a 4% increase in consumer spending - the highest since last year. The Dow Jones Industrial Average and Nasdaq Composite declined over 0.76% and 1% respectively, reflecting investor concerns over an anticipated economic slowdown. In the midst of these market fluctuations, tech giants Alphabet (NASDAQ:GOOGL) and Meta Platforms (NASDAQ:META) fell short of impressing investors, despite exceeding revenue and earnings targets. Meta Platforms, in particular, suffered a 3.7% stock drop after warning of ""volatility"" in the ad market. This news came as a surprise to many investors, given the company's recent surge in quarterly profit. The market also responded poorly to Alphabet and Tesla (NASDAQ:TSLA), amplifying investor unease. Toymakers Hasbro (NASDAQ:HAS) and Mattel (NASDAQ:MAT) saw their shares fall, while drugmaker Merck and IBM (NYSE:IBM) experienced gains. Commodity prices followed suit with the stock market trend, with West Texas Intermediate and Brent crude oil both decreasing by 2.3%. Bitcoin also slipped by 1.6%, and the 10-year yield fell by 10.6 basis points. Despite the strong Q3 GDP growth, Wall Street's reaction underscores the complex interplay between economic indicators and investor sentiment. This article was generated with the support of AI and reviewed by an editor. For more information see our T&C."
2,"Wall Street ends lower on mixed earnings, robust data",Reuters,2023-10-26 05:48:00,https://www.investing.com/news/economy/nasdaq-futures-slide-as-megacaps-drag-economic-data-in-focus-3209627,"By Stephen Culp NEW YORK (Reuters) - U.S. stocks tumbled on Thursday, dragged by tech and tech-adjacent megacap shares as investors digested mixed quarterly earnings and signs of economic resiliency that could encourage the Federal Reserve to keep interest rates at a restrictive level longer than expected. All three major U.S. stock indexes ended in the red, and all remain on track for weekly declines. The tech-heavy Nasdaq suffered the biggest percentage drop, weighed down by the ""magnificent seven"" group of megacap stocks in the face of cloudy earnings guidance and the ""higher for longer"" interest rate scenario. The NYSE FANG+ index of momentum stocks closed down 2.7%. ""Today is all about the 'magnificent seven' and I don't think there's anything they coul

In [ ]:
#####now run the function that will collect articles about multiple stocks at once.
#stock_news_collect_list(stock_list,pages)
#stock_list: a list data type containing stock names or tickers
#pages: the number of pages you want to collect news articles from (if pages > then the number of pages actually available, pages will be the max number instead)

In [ ]:
stock_news_collect_list(['apple','EA','Tesla'],3)

Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...


'Just finished apple, page 1.'

Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...


'Just finished apple, page 2.'

Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...
Collected the text for an article about apple. Now pausing for 15 seconds...


...

Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...


'Just finished Tesla, page 2.'

Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...
Collected the text for an article about Tesla. Now pausing for 15 seconds...


'Just finished Tesla, page 3.'

,stock_name,stock_tick,stock_country,stock_ex,art_name,art_source,art_date,art_link,art_text
0,Apple Inc,AAPL,USA,Stock - NASDAQ equities,Cinemark anticipates robust recovery with $3 b...,Investing.com,2023-11-06 10:26:00,https://www.investing.com/news/stock-market-ne...,Cinemark Holdings (NYSE:CNK) Inc. has reporte...
1,Apple Inc,AAPL,USA,Stock - NASDAQ equities,Hedge funds pile into US stock rally at fastes...,Reuters,2023-11-06 07:23:00,https://www.investing.com/news/economy/hedge-f...,By Nell Mackenzie LONDON (Reuters) - Hedge fu...
2,Apple Inc,AAPL,USA,Stock - NASDAQ equities,Berkshire Hathaway registers Q3 loss amid stro...,Investing.com,2023-11-06 05:59:00,https://www.investing.com/news/stock-market-ne...,Warren Buffett's Berkshire Hathaway (NYSE:BRK...
3,Apple Inc,AAPL,USA,Stock - NASDAQ equities,"U.S. smartphone market sees decline in Q3, fol...",Investing.com,2023-11-05 07:59:00,https://www.investing.com/news/stock-market-ne...,"In the third quarter of 2023, the US smartpho..."
4,Apple Inc,AAPL,USA,Stock - NASDAQ equities,Apple's wishy-washy guidance; AMD's AI optimis...,Investing.com,2023-11-05 04:45:00,https://www.investing.com/news/stock-market-ne...,By Louis Juricic and Sarina Isaacs Investing....
...,...,...,...,...,...,...,...,...,...
64,Tesla Inc,TSLA,USA,Stock - NASDAQ equities,Tesla's Oct China-made EV sales fall 2.6% from...,Reuters,2023-11-02 05:08:00,https://www.investing.com/news/stock-market-ne...,BEIJING/SHANGHAI (Reuters) - U.S. automaker T...
65,Tesla Inc,TSLA,USA,Stock - NASDAQ equities,Tesla shares plummet amid concerns of dwindlin...,Investing.com,2023-11-02 03:03:00,https://www.investing.com/news/stock-market-ne...,Tesla (NASDAQ:TSLA)'s stock has taken a sharp...
66,Tesla Inc,TSLA,USA,Stock - NASDAQ equities,"US stocks rally as Fed holds rates, prompts ho...",Reuters,2023-11-01 05:51:00,https://www.investing.com/news/economy/futures...,By Sinéad Carew and Amruta Khandekar (Reuters...
67,Tesla Inc,TSLA,USA,Stock - NASDAQ equities,Cryptocurrencies Face Downturn While Select St...,Investing.com,2023-11-01 12:49:00,https://www.investing.com/news/cryptocurrency-...,"In Wednesday's morning trading session, a maj..."
